In [2]:
# -*- coding: utf-8 -*-
# Matriz de Criticidad de AIBs (fuera de Power BI) - versión cx_Oracle
# Lee Oracle + Excel EXPOSICIÓN, aplica las mismas transformaciones de Power Query (M)
# y clona las medidas DAX compartidas. Exporta Matriz_AIB_export.xlsx.

from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import cx_Oracle

# ================== CREDENCIALES / RUTAS (texto plano) ==================
ORA_USER    = "RY16123"
ORA_PASS    = "Luciano284"
ORA_HOST    = "slplpgmoora03"
ORA_PORT    = 1527
ORA_SERVICE = "psfu"

EXPO_PATH   = r"C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx"
EXPO_SHEET  = "EXPOSICION"   # nombre de hoja/tabla dentro del Excel

# ================== QUERIES (según tu M) ==================
SQL_FDP_DINA = """
SELECT DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_1,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_2,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5,
       FIC_ULTIMO_DINAMOMETRO.FECHA_HORA,
       FIC_ULTIMO_DINAMOMETRO.REALIZADO_POR,
       FIC_ULTIMO_DINAMOMETRO.AIB_MARCA_Y_DESC_API,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBEB_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_DISPONIBLE,
       FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100 AS LLENAD_BOMBA_PCT,
       FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100 AS ESTRUCTURA_PCT,
       VTOW_WELL_LAST_CONTROL_DET.PROD_OIL,
       VTOW_WELL_LAST_CONTROL_DET.PROD_GAS,
       VTOW_WELL_LAST_CONTROL_DET.PROD_WAT,
       FIC_ULTIMO_DINAMOMETRO.AIB_GPM,
       FIC_ULTIMO_DINAMOMETRO.AIB_DIAMETRO_POLEA_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.MOTOR_DIAMETRO_POLEA
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
"""

# Consulta "Producción" de tu M: UPPER(NIVEL_4), SUM(PROD_OIL) agrupado por NIVEL_4
SQL_PRODUCCION = """
SELECT UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4) AS NIVEL_4,
       SUM(VTOW_WELL_LAST_CONTROL_DET.PROD_OIL) AS PROD_OIL_1
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
GROUP BY UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4)
"""

# ================== CONEXIÓN ORACLE (cx_Oracle) ==================
def get_connection():
    dsn = cx_Oracle.makedsn(ORA_HOST, ORA_PORT, service_name=ORA_SERVICE)
    return cx_Oracle.connect(ORA_USER, ORA_PASS, dsn, encoding="UTF-8")

# ================== EXPO (traducción de tu M) ==================
def load_exposicion(path: str, sheet: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet)
    if "Cantidad" in df.columns:
        df = df.drop(columns=["Cantidad"])
    df["Denominacion API"] = df["Denominacion API"].astype(str).str.upper().str.strip()
    # Asegurar columnas esperadas
    for c in ["Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]:
        if c not in df.columns:
            df[c] = np.nan
    return df

# ================== FDP_DINA y Producción (via cx_Oracle) ==================
def read_sql_df(conn, sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, con=conn)

def load_fdp(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_FDP_DINA)
    # Text.Upper(NIVEL_4)
    df["NIVEL_4"] = df["NIVEL_4"].astype(str).str.upper()
    return df

def load_produccion(conn) -> pd.DataFrame:
    return read_sql_df(conn, SQL_PRODUCCION)  # columnas: NIVEL_4, PROD_OIL_1

# ================== Transformaciones equivalentes a Power Query ==================
def join_expo(df_fdp: pd.DataFrame, expo: pd.DataFrame) -> pd.DataFrame:
    df = df_fdp.copy()
    df["__API_KEY__"] = df["AIB_MARCA_Y_DESC_API"].astype(str).str.upper().str.strip()
    ex = expo.copy()
    ex["__API_KEY__"] = ex["Denominacion API"]

    df = df.merge(
        ex[["__API_KEY__","Denominacion API","Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]],
        how="left", on="__API_KEY__"
    ).drop(columns=["__API_KEY__"])

    # Renombres como en Power Query (prefijo EXPOSICIÓN.)
    df = df.rename(columns={
        "Denominacion API": "EXPOSICIÓN.Denominacion API",
        "Marca": "EXPOSICIÓN.Marca",
        "Tipo": "EXPOSICIÓN.Tipo",
        "Torque max": "EXPOSICIÓN.Torque max",
        "SENSIBILIDAD CR": "EXPOSICIÓN.SENSIBILIDAD CR",
        "SENSIBILIDAD EST": "EXPOSICIÓN.SENSIBILIDAD EST",
    })

    # ID = NOMBRE_CORTO & NIVEL_5 → filtrar nulos/vacíos, quitar duplicados por ID
    df["ID_tmp"] = (df["NOMBRE_CORTO"].fillna("").astype(str) + df["NIVEL_5"].fillna("").astype(str))
    df = df[(df["ID_tmp"]!="") & df["NOMBRE_CORTO"].notna()]
    df = df.drop_duplicates(subset=["ID_tmp"]).drop(columns=["ID_tmp"])

    # Reemplazos de nulos por 0 en torques
    for c in ["AIBRR_TORQUE_MAXIMO_REDUCTOR","AIBEB_TORQUE_MAXIMO_REDUCTOR"]:
        df[c] = df[c].fillna(0)

    # Reemplazo Tierra del Fuego → Chubut
    df["NIVEL_2"] = df["NIVEL_2"].replace({"Tierra del Fuego": "Chubut"})

    return df

def attach_produccion(df: pd.DataFrame, prod: pd.DataFrame) -> pd.DataFrame:
    return df.merge(prod, how="left", on="NIVEL_4")

# ================== Medidas/derivadas (clon DAX) ==================
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    # Ratio_Produccion
    d["PROD_OIL_1"] = d["PROD_OIL_1"].replace({0: np.nan})
    d["Ratio_Produccion"] = d["PROD_OIL"] / d["PROD_OIL_1"]

    # CONSECUENCIA (texto por cortes)
    def seg_consecuencia(x):
        if pd.isna(x): return "0 - 0,0025"
        if x >= 0.0065: return "Mayor a 0,0065"
        if x >= 0.0025: return "0,0025 - 0,0065"
        return "0 - 0,0025"
    d["f_CONSECUENCIA"] = d["Ratio_Produccion"].apply(seg_consecuencia)
    d["CONSECUENCIA"]   = d["f_CONSECUENCIA"]

    # TORQUE_BAL[%] y TORQUE[%]
    d["TORQUE_BAL[%]"] = d["AIBEB_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]
    d["TORQUE[%]"]     = d["AIBRR_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]

    # Factores y EXIGENCIA (según tu M)
    d["f_CR"]  = np.select([d["TORQUE[%]"] > 1, d["TORQUE[%]"] >= 0.85], [10, 3.7], default=0)
    d["f_CR3"] = d["EXPOSICIÓN.SENSIBILIDAD CR"].map({"ALTA":3, "MEDIA":1.5}).fillna(1.0)
    d["CAJA REDUCTORA"]  = d["f_CR"] * d["f_CR3"]
    d["CAJA REDUCTORA_"] = np.where(d["TORQUE_BAL[%]"]>1, 1.5*d["CAJA REDUCTORA"], d["CAJA REDUCTORA"])

    d["f_estr"]  = np.select([d["ESTRUCTURA_PCT"] >= 0.95, d["ESTRUCTURA_PCT"] > 0.85], [20,5], default=1)
    d["f_estr3"] = d["EXPOSICIÓN.SENSIBILIDAD EST"].map({"ALTA":3,"MEDIA":1.5}).fillna(1.0)
    d["ESTRUCTURA"] = d["f_estr"] * d["f_estr3"]

    d["f_GPM"]  = np.select([d["AIB_GPM"] > 9, d["AIB_GPM"] > 7], [3, 1.5], default=1)
    d["f_GPM2"] = np.where(d["LLENAD_BOMBA_PCT"] < 0.75, d["f_GPM"]*2, d["f_GPM"])

    d["EXIGENCIA"]   = d["f_GPM2"] * d["ESTRUCTURA"] + d["CAJA REDUCTORA_"]
    d["F_EXIGENCIA"] = np.select([d["EXIGENCIA"]>10, d["EXIGENCIA"]>4], ["A","M"], default="B")

    # CRITICIDAD (reglas de tu M)
    def crit(row):
        rp = row["Ratio_Produccion"]; fe = row["F_EXIGENCIA"]
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "CRITICO"
        if rp >= 0.0025 and fe == "A": return "ALERTA"
        if rp <  0.0025 and fe == "A": return "ALERTA"
        if rp >= 0.0065 and fe == "M": return "ALERTA"
        if rp >= 0.0025 and fe == "M": return "ALERTA"
        if rp <  0.0025 and fe == "M": return "ALERTA"
        return "NORMAL"
    d["CRITICIDAD"] = d.apply(crit, axis=1).astype(str)
    d["ref_criticidad"] = d["CRITICIDAD"].map({"CRITICO":1, "ALERTA":2, "NORMAL":3}).astype("Int64")

    # Color Detalle CRITICIDAD (DAX)
    def color_detalle(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return ""
        if rp >= 0.0065 and fe == "A": return "RED"
        if rp >= 0.0025 and fe == "A": return "ORANGE"
        if rp <  0.0025 and fe == "A": return "ORANGE"
        if rp >= 0.0065 and fe == "M": return "YELLOW"
        if rp >= 0.0025 and fe == "M": return "YELLOW"
        if rp <  0.0025 and fe == "M": return "YELLOW"
        if rp >= 0.0065 and fe == "B": return "GREEN"
        if rp >= 0.0025 and fe == "B": return "GREEN"
        if rp <  0.0025 and fe == "B": return "GREEN"
        return ""
    d["Color Detalle CRITICIDAD"] = np.vectorize(color_detalle)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    # criticidad color / texto (DAX)
    d["criticidad color"] = d["CRITICIDAD"].map({"CRITICO":"Red", "ALERTA":"YELLOW", "NORMAL":"Green"}).fillna("White")
    d["criticidad texto"] = d["CRITICIDAD"].map({"CRITICO":"white", "ALERTA":"black", "NORMAL":"white"}).fillna("White")

    # DETALLE CRITICIDAD (DAX)
    def detalle_crit(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "AA"
        if rp >= 0.0025 and fe == "A": return "AM"
        if rp <  0.0025 and fe == "A": return "AB"
        if rp >= 0.0065 and fe == "M": return "MA"
        if rp >= 0.0025 and fe == "M": return "MM"
        if rp <  0.0025 and fe == "M": return "MB"
        if rp >= 0.0065 and fe == "B": return "BA"
        if rp >= 0.0025 and fe == "B": return "BM"
        if rp <  0.0025 and fe == "B": return "BB"
        return "NORMAL"
    d["DETALLE CRITICIDAD"] = np.vectorize(detalle_crit)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    # Días sin dina = DATEDIFF(MAX(FECHA_HORA), TODAY(), DAY) → MAX por pozo
    d["FECHA_HORA"] = pd.to_datetime(d["FECHA_HORA"], errors="coerce")
    max_por_pozo = d.groupby("NOMBRE_CORTO", dropna=False)["FECHA_HORA"].max().rename("FECHA_MAX").reset_index()
    d = d.merge(max_por_pozo, on="NOMBRE_CORTO", how="left")
    d["Días sin dina"] = (pd.Timestamp.today().normalize() - d["FECHA_MAX"].dt.normalize()).dt.days

    # Columnas calculadas dentro de FDP_DINA
    d["Correlativo Exigencia"] = d["F_EXIGENCIA"].map({"A":"10 +", "M":"04-10", "B":"0-04"}).fillna("")
    d["ID"] = d["f_CONSECUENCIA"].fillna("").astype(str) + d["Correlativo Exigencia"].fillna("").astype(str)

    # Filtro final igual que M
    d = d[(d["NOMBRE_CORTO"].notna()) & (d["NOMBRE_CORTO"].astype(str)!="")]

    return d

# ================== PIPELINE ==================
def main():
    print("Conectando a Oracle (cx_Oracle)…")
    with get_connection() as conn:
        print("Leyendo FDP_DINA…")
        fdp = load_fdp(conn)

        print("Leyendo Producción (sum PROD_OIL por NIVEL_4)…")
        prod = load_produccion(conn)

    print("Leyendo Excel EXPOSICIÓN…")
    expo = load_exposicion(EXPO_PATH, EXPO_SHEET)

    print("Aplicando joins/limpieza (EXPOSICIÓN)…")
    fdp = join_expo(fdp, expo)

    print("Adjuntando Producción…")
    fdp = attach_produccion(fdp, prod)

    print("Calculando métricas / criticidad…")
    final = add_features(fdp)

    out = Path("Matriz_AIB_export.xlsx")
    final.to_excel(out, index=False)
    print(f"✅ Exportado {len(final):,} filas → {out.absolute()}")

if __name__ == "__main__":
    main()


Conectando a Oracle (cx_Oracle)…
Leyendo FDP_DINA…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\944859134.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo Producción (sum PROD_OIL por NIVEL_4)…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\944859134.py:90: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo Excel EXPOSICIÓN…


ValueError: Worksheet named 'EXPOSICION' not found

In [3]:
# -*- coding: utf-8 -*-
# Matriz de Criticidad de AIBs (fuera de Power BI) - versión cx_Oracle
# Lee Oracle + Excel EXPOSICIÓN, aplica transformaciones equivalentes a Power Query (M)
# y clona las medidas DAX que compartiste. Exporta Matriz_AIB_export.xlsx.

from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import cx_Oracle

# ================== CREDENCIALES / RUTAS (texto plano, como pediste) ==================
ORA_USER    = "RY16123"
ORA_PASS    = "Luciano284"
ORA_HOST    = "slplpgmoora03"
ORA_PORT    = 1527
ORA_SERVICE = "psfu"

EXPO_PATH   = r"C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx"
EXPO_SHEET  = None  # <- auto-detección de hoja (dejalo None). Si querés fijar, poné el nombre exacto.

# ================== QUERIES (según tu M) ==================
SQL_FDP_DINA = """
SELECT DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_1,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_2,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5,
       FIC_ULTIMO_DINAMOMETRO.FECHA_HORA,
       FIC_ULTIMO_DINAMOMETRO.REALIZADO_POR,
       FIC_ULTIMO_DINAMOMETRO.AIB_MARCA_Y_DESC_API,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBEB_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_DISPONIBLE,
       FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100 AS LLENAD_BOMBA_PCT,
       FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100 AS ESTRUCTURA_PCT,
       VTOW_WELL_LAST_CONTROL_DET.PROD_OIL,
       VTOW_WELL_LAST_CONTROL_DET.PROD_GAS,
       VTOW_WELL_LAST_CONTROL_DET.PROD_WAT,
       FIC_ULTIMO_DINAMOMETRO.AIB_GPM,
       FIC_ULTIMO_DINAMOMETRO.AIB_DIAMETRO_POLEA_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.MOTOR_DIAMETRO_POLEA
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
"""

# Consulta "Producción" de tu M: UPPER(NIVEL_4), SUM(PROD_OIL) agrupado por NIVEL_4
SQL_PRODUCCION = """
SELECT UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4) AS NIVEL_4,
       SUM(VTOW_WELL_LAST_CONTROL_DET.PROD_OIL) AS PROD_OIL_1
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
GROUP BY UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4)
"""

# ================== CONEXIÓN ORACLE (cx_Oracle) ==================
def get_connection():
    dsn = cx_Oracle.makedsn(ORA_HOST, ORA_PORT, service_name=ORA_SERVICE)
    return cx_Oracle.connect(ORA_USER, ORA_PASS, dsn, encoding="UTF-8")

def read_sql_df(conn, sql: str) -> pd.DataFrame:
    # pandas muestra un warning (recomienda SQLAlchemy), pero funciona igual con cx_Oracle.
    return pd.read_sql(sql, con=conn)

# ================== EXPO (auto-detección de hoja) ==================
def load_exposicion(path: str, sheet: str | None) -> pd.DataFrame:
    """
    Carga el Excel de EXPOSICIÓN aunque la hoja no se llame 'EXPOSICION'.
    Estrategia:
      1) Si sheet está definido y existe -> lo usa.
      2) Prueba nombres comunes.
      3) Recorre todas las hojas y elige la que tenga la columna 'Denominacion API'.
    """
    xls = pd.ExcelFile(path, engine="openpyxl")
    print(f"Hojas encontradas en '{path}': {xls.sheet_names}")

    # 1) Hoja indicada por el usuario
    if sheet and sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
    else:
        # 2) Intentar nombres típicos
        candidatos = ["EXPOSICION", "EXPOSICIÓN", "Exposicion", "Hoja1", "Sheet1"]
        df = None
        for s in candidatos:
            if s in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=s)
                break
        # 3) Buscar por columna 'Denominacion API'
        if df is None:
            for s in xls.sheet_names:
                tmp = pd.read_excel(xls, sheet_name=s)
                cols_norm = [str(c).strip().lower() for c in tmp.columns]
                if "denominacion api" in cols_norm:
                    df = tmp
                    print(f"Usando hoja detectada por columna: {s}")
                    break
        if df is None:
            raise ValueError(
                f"No pude encontrar la hoja de EXPOSICIÓN. "
                f"Indicá EXPO_SHEET con el nombre exacto. Hojas: {xls.sheet_names}"
            )

    # Quitar 'Cantidad' si existe
    if "Cantidad" in df.columns:
        df = df.drop(columns=["Cantidad"])

    # Normalizar columna clave (Denominacion API)
    col_api = None
    for c in df.columns:
        if str(c).strip().lower() == "denominacion api":
            col_api = c
            break
    if col_api is None:
        raise ValueError("No encuentro la columna 'Denominacion API' en la hoja seleccionada.")

    df[col_api] = df[col_api].astype(str).str.upper().str.strip()
    df = df.rename(columns={col_api: "Denominacion API"})

    # Asegurar resto de columnas esperadas
    for c in ["Marca", "Tipo", "Torque max", "SENSIBILIDAD CR", "SENSIBILIDAD EST"]:
        if c not in df.columns:
            df[c] = np.nan

    return df

# ================== FDP_DINA y Producción ==================
def load_fdp(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_FDP_DINA)
    # Text.Upper(NIVEL_4)
    df["NIVEL_4"] = df["NIVEL_4"].astype(str).str.upper()
    return df

def load_produccion(conn) -> pd.DataFrame:
    return read_sql_df(conn, SQL_PRODUCCION)  # columnas: NIVEL_4, PROD_OIL_1

# ================== Transformaciones equivalentes a Power Query ==================
def join_expo(df_fdp: pd.DataFrame, expo: pd.DataFrame) -> pd.DataFrame:
    df = df_fdp.copy()
    df["__API_KEY__"] = df["AIB_MARCA_Y_DESC_API"].astype(str).str.upper().str.strip()
    ex = expo.copy()
    ex["__API_KEY__"] = ex["Denominacion API"]

    df = df.merge(
        ex[["__API_KEY__","Denominacion API","Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]],
        how="left", on="__API_KEY__"
    ).drop(columns=["__API_KEY__"])

    # Prefijo EXPOSICIÓN.
    df = df.rename(columns={
        "Denominacion API": "EXPOSICIÓN.Denominacion API",
        "Marca": "EXPOSICIÓN.Marca",
        "Tipo": "EXPOSICIÓN.Tipo",
        "Torque max": "EXPOSICIÓN.Torque max",
        "SENSIBILIDAD CR": "EXPOSICIÓN.SENSIBILIDAD CR",
        "SENSIBILIDAD EST": "EXPOSICIÓN.SENSIBILIDAD EST",
    })

    # ID = NOMBRE_CORTO & NIVEL_5 → filtrar y deduplicar
    df["ID_tmp"] = (df["NOMBRE_CORTO"].fillna("").astype(str) + df["NIVEL_5"].fillna("").astype(str))
    df = df[(df["ID_tmp"]!="") & df["NOMBRE_CORTO"].notna()]
    df = df.drop_duplicates(subset=["ID_tmp"]).drop(columns=["ID_tmp"])

    # Reemplazos de nulos por 0 en torques
    for c in ["AIBRR_TORQUE_MAXIMO_REDUCTOR","AIBEB_TORQUE_MAXIMO_REDUCTOR"]:
        df[c] = df[c].fillna(0)

    # Reemplazo Tierra del Fuego → Chubut
    df["NIVEL_2"] = df["NIVEL_2"].replace({"Tierra del Fuego": "Chubut"})

    return df

def attach_produccion(df: pd.DataFrame, prod: pd.DataFrame) -> pd.DataFrame:
    return df.merge(prod, how="left", on="NIVEL_4")

# ================== Medidas/derivadas (clon DAX) ==================
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    # Ratio_Produccion
    d["PROD_OIL_1"] = d["PROD_OIL_1"].replace({0: np.nan})
    d["Ratio_Produccion"] = d["PROD_OIL"] / d["PROD_OIL_1"]

    # CONSECUENCIA (texto por cortes)
    def seg_consecuencia(x):
        if pd.isna(x): return "0 - 0,0025"
        if x >= 0.0065: return "Mayor a 0,0065"
        if x >= 0.0025: return "0,0025 - 0,0065"
        return "0 - 0,0025"
    d["f_CONSECUENCIA"] = d["Ratio_Produccion"].apply(seg_consecuencia)
    d["CONSECUENCIA"]   = d["f_CONSECUENCIA"]

    # TORQUE_BAL[%] y TORQUE[%]
    d["TORQUE_BAL[%]"] = d["AIBEB_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]
    d["TORQUE[%]"]     = d["AIBRR_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]

    # Factores y EXIGENCIA (según tu M)
    d["f_CR"]  = np.select([d["TORQUE[%]"] > 1, d["TORQUE[%]"] >= 0.85], [10, 3.7], default=0)
    d["f_CR3"] = d["EXPOSICIÓN.SENSIBILIDAD CR"].map({"ALTA":3, "MEDIA":1.5}).fillna(1.0)
    d["CAJA REDUCTORA"]  = d["f_CR"] * d["f_CR3"]
    d["CAJA REDUCTORA_"] = np.where(d["TORQUE_BAL[%]"]>1, 1.5*d["CAJA REDUCTORA"], d["CAJA REDUCTORA"])

    d["f_estr"]  = np.select([d["ESTRUCTURA_PCT"] >= 0.95, d["ESTRUCTURA_PCT"] > 0.85], [20,5], default=1)
    d["f_estr3"] = d["EXPOSICIÓN.SENSIBILIDAD EST"].map({"ALTA":3,"MEDIA":1.5}).fillna(1.0)
    d["ESTRUCTURA"] = d["f_estr"] * d["f_estr3"]

    d["f_GPM"]  = np.select([d["AIB_GPM"] > 9, d["AIB_GPM"] > 7], [3, 1.5], default=1)
    d["f_GPM2"] = np.where(d["LLENAD_BOMBA_PCT"] < 0.75, d["f_GPM"]*2, d["f_GPM"])

    d["EXIGENCIA"]   = d["f_GPM2"] * d["ESTRUCTURA"] + d["CAJA REDUCTORA_"]
    d["F_EXIGENCIA"] = np.select([d["EXIGENCIA"]>10, d["EXIGENCIA"]>4], ["A","M"], default="B")

    # CRITICIDAD (reglas de tu M)
    def crit(row):
        rp = row["Ratio_Produccion"]; fe = row["F_EXIGENCIA"]
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "CRITICO"
        if rp >= 0.0025 and fe == "A": return "ALERTA"
        if rp <  0.0025 and fe == "A": return "ALERTA"
        if rp >= 0.0065 and fe == "M": return "ALERTA"
        if rp >= 0.0025 and fe == "M": return "ALERTA"
        if rp <  0.0025 and fe == "M": return "ALERTA"
        return "NORMAL"
    d["CRITICIDAD"] = d.apply(crit, axis=1).astype(str)
    d["ref_criticidad"] = d["CRITICIDAD"].map({"CRITICO":1, "ALERTA":2, "NORMAL":3}).astype("Int64")

    # Color Detalle CRITICIDAD (DAX)
    def color_detalle(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return ""
        if rp >= 0.0065 and fe == "A": return "RED"
        if rp >= 0.0025 and fe == "A": return "ORANGE"
        if rp <  0.0025 and fe == "A": return "ORANGE"
        if rp >= 0.0065 and fe == "M": return "YELLOW"
        if rp >= 0.0025 and fe == "M": return "YELLOW"
        if rp <  0.0025 and fe == "M": return "YELLOW"
        if rp >= 0.0065 and fe == "B": return "GREEN"
        if rp >= 0.0025 and fe == "B": return "GREEN"
        if rp <  0.0025 and fe == "B": return "GREEN"
        return ""
    d["Color Detalle CRITICIDAD"] = np.vectorize(color_detalle)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    # criticidad color / texto (DAX)
    d["criticidad color"] = d["CRITICIDAD"].map({"CRITICO":"Red", "ALERTA":"YELLOW", "NORMAL":"Green"}).fillna("White")
    d["criticidad texto"] = d["CRITICIDAD"].map({"CRITICO":"white", "ALERTA":"black", "NORMAL":"white"}).fillna("White")

    # DETALLE CRITICIDAD (DAX)
    def detalle_crit(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "AA"
        if rp >= 0.0025 and fe == "A": return "AM"
        if rp <  0.0025 and fe == "A": return "AB"
        if rp >= 0.0065 and fe == "M": return "MA"
        if rp >= 0.0025 and fe == "M": return "MM"
        if rp <  0.0025 and fe == "M": return "MB"
        if rp >= 0.0065 and fe == "B": return "BA"
        if rp >= 0.0025 and fe == "B": return "BM"
        if rp <  0.0025 and fe == "B": return "BB"
        return "NORMAL"
    d["DETALLE CRITICIDAD"] = np.vectorize(detalle_crit)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    # Días sin dina = DATEDIFF(MAX(FECHA_HORA), TODAY(), DAY) → MAX por pozo
    d["FECHA_HORA"] = pd.to_datetime(d["FECHA_HORA"], errors="coerce")
    max_por_pozo = d.groupby("NOMBRE_CORTO", dropna=False)["FECHA_HORA"].max().rename("FECHA_MAX").reset_index()
    d = d.merge(max_por_pozo, on="NOMBRE_CORTO", how="left")
    d["Días sin dina"] = (pd.Timestamp.today().normalize() - d["FECHA_MAX"].dt.normalize()).dt.days

    # Columnas calculadas dentro de FDP_DINA
    d["Correlativo Exigencia"] = d["F_EXIGENCIA"].map({"A":"10 +", "M":"04-10", "B":"0-04"}).fillna("")
    d["ID"] = d["f_CONSECUENCIA"].fillna("").astype(str) + d["Correlativo Exigencia"].fillna("").astype(str)

    # Filtro final igual que M
    d = d[(d["NOMBRE_CORTO"].notna()) & (d["NOMBRE_CORTO"].astype(str)!="")]

    return d

# ================== PIPELINE ==================
def main():
    print("Conectando a Oracle (cx_Oracle)…")
    with get_connection() as conn:
        print("Leyendo FDP_DINA…")
        fdp = load_fdp(conn)

        print("Leyendo Producción (sum PROD_OIL por NIVEL_4)…")
        prod = load_produccion(conn)

    print("Leyendo Excel EXPOSICIÓN…")
    expo = load_exposicion(EXPO_PATH, EXPO_SHEET)

    print("Aplicando joins/limpieza (EXPOSICIÓN)…")
    fdp = join_expo(fdp, expo)

    print("Adjuntando Producción…")
    fdp = attach_produccion(fdp, prod)

    print("Calculando métricas / criticidad…")
    final = add_features(fdp)

    out = Path("Matriz_AIB_export.xlsx")
    final.to_excel(out, index=False)
    print(f"✅ Exportado {len(final):,} filas → {out.absolute()}")

if __name__ == "__main__":
    main()


Conectando a Oracle (cx_Oracle)…
Leyendo FDP_DINA…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\1230803187.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo Producción (sum PROD_OIL por NIVEL_4)…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\1230803187.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo Excel EXPOSICIÓN…
Hojas encontradas en 'C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx': ['Hoja5', 'EXPOSICIÓN', 'Hoja1', 'Hoja2', 'IDP', 'Hoja3', 'OBSERVACIONES']
Aplicando joins/limpieza (EXPOSICIÓN)…
Adjuntando Producción…
Calculando métricas / criticidad…
✅ Exportado 3,767 filas → C:\Users\ry16123\Matriz_AIB_export.xlsx


In [4]:
# -*- coding: utf-8 -*-
# Matriz de Criticidad de AIBs (fuera de Power BI) - cx_Oracle
# Hoja 1: columnas solicitadas de FDP_DINA transformada
# Hoja 2: consulta ACCIONES

from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import cx_Oracle

# ================== CREDENCIALES / RUTAS ==================
ORA_USER    = "RY16123"
ORA_PASS    = "Luciano284"
ORA_HOST    = "slplpgmoora03"
ORA_PORT    = 1527
ORA_SERVICE = "psfu"

EXPO_PATH   = r"C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx"
EXPO_SHEET  = None  # auto-detección de hoja

# ================== QUERIES ==================
SQL_FDP_DINA = """
SELECT DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_1,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_2,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5,
       FIC_ULTIMO_DINAMOMETRO.FECHA_HORA,
       FIC_ULTIMO_DINAMOMETRO.REALIZADO_POR,
       FIC_ULTIMO_DINAMOMETRO.AIB_MARCA_Y_DESC_API,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBEB_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_DISPONIBLE,
       FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100 AS LLENAD_BOMBA_PCT,
       FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100 AS ESTRUCTURA_PCT,
       VTOW_WELL_LAST_CONTROL_DET.PROD_OIL,
       VTOW_WELL_LAST_CONTROL_DET.PROD_GAS,
       VTOW_WELL_LAST_CONTROL_DET.PROD_WAT,
       FIC_ULTIMO_DINAMOMETRO.AIB_GPM,
       FIC_ULTIMO_DINAMOMETRO.AIB_DIAMETRO_POLEA_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.MOTOR_DIAMETRO_POLEA
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
"""

SQL_PRODUCCION = """
SELECT UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4) AS NIVEL_4,
       SUM(VTOW_WELL_LAST_CONTROL_DET.PROD_OIL) AS PROD_OIL_1
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
GROUP BY UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4)
"""

SQL_ACCIONES = """
SELECT DISTINCT
  DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
  FIC_ACCIONES.ACTIVIDAD,
  FIC_ACCIONES.ESTADO,
  FIC_ACCIONES."ESTADO AUTORIZACIÓN",
  FIC_ACCIONES.FECHAACCION,
  FIC_ACCIONES."FECHA AUTORIZACION",
  FIC_ACCIONES.FECHAREALIZACION,
  FIC_ACCIONES.JUSTIFICACION,
  FIC_ACCIONES.OBJETIVO,
  FIC_ACCIONES.OBSERVACION,
  FIC_ACCIONES.ORIGEN,
  FIC_ACCIONES."INCREMENTO BRUTA",
  FIC_ACCIONES."INCREMENTO PETRÓLEO",
  FIC_ACCIONES.RECURSO,
  FIC_ACCIONES.SUBACTIVIDAD,
  FIC_ACCIONES."USUARIO AUTORIZANTE",
  FIC_ACCIONES."USUARIO CREADOR",
  DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_POZO,
  DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO_POZO
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.FIC_ACCIONES FIC_ACCIONES
WHERE (FIC_ACCIONES.CLAVEPOZO = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
  AND (DBU_FIC_ORG_ESTRUCTURAL.COMP_SK = FIC_ACCIONES.CLAVEPOZO)
  AND (FIC_ACCIONES.FECHAACCION >= TO_DATE('20230101000000','YYYYMMDDHH24MISS'))
  AND (DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3 IN ('Las Heras CG - Canadon Escondida','Los Perales','El Guadal','Seco Leon - Pico Truncado'))
"""

# ================== CONEXIÓN ORACLE (cx_Oracle) ==================
def get_connection():
    dsn = cx_Oracle.makedsn(ORA_HOST, ORA_PORT, service_name=ORA_SERVICE)
    return cx_Oracle.connect(ORA_USER, ORA_PASS, dsn, encoding="UTF-8")

def read_sql_df(conn, sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, con=conn)  # warning inofensivo

# ================== EXPO (auto-detección de hoja) ==================
def load_exposicion(path: str, sheet: str | None) -> pd.DataFrame:
    xls = pd.ExcelFile(path, engine="openpyxl")
    print(f"Hojas encontradas en '{path}': {xls.sheet_names}")

    if sheet and sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
    else:
        candidatos = ["EXPOSICION", "EXPOSICIÓN", "Exposicion", "Hoja1", "Sheet1"]
        df = None
        for s in candidatos:
            if s in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=s)
                break
        if df is None:
            for s in xls.sheet_names:
                tmp = pd.read_excel(xls, sheet_name=s)
                cols_norm = [str(c).strip().lower() for c in tmp.columns]
                if "denominacion api" in cols_norm:
                    df = tmp
                    print(f"Usando hoja detectada por columna: {s}")
                    break
        if df is None:
            raise ValueError(f"No pude encontrar la hoja de EXPOSICIÓN. Hojas: {xls.sheet_names}")

    if "Cantidad" in df.columns:
        df = df.drop(columns=["Cantidad"])

    # Normalizar columna clave
    col_api = None
    for c in df.columns:
        if str(c).strip().lower() == "denominacion api":
            col_api = c; break
    if col_api is None:
        raise ValueError("No encuentro la columna 'Denominacion API' en la hoja seleccionada.")

    df[col_api] = df[col_api].astype(str).str.upper().str.strip()
    df = df.rename(columns={col_api: "Denominacion API"})

    for c in ["Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]:
        if c not in df.columns:
            df[c] = np.nan
    return df

# ================== Cargas base ==================
def load_fdp(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_FDP_DINA)
    df["NIVEL_4"] = df["NIVEL_4"].astype(str).str.upper()
    return df

def load_produccion(conn) -> pd.DataFrame:
    return read_sql_df(conn, SQL_PRODUCCION)  # NIVEL_4, PROD_OIL_1

def load_acciones(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_ACCIONES)
    # opcional: convertir fechas a datetime
    for c in ["FECHAACCION","FECHAREALIZACION","FECHA AUTORIZACION"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

# ================== Transformaciones ==================
def join_expo(df_fdp: pd.DataFrame, expo: pd.DataFrame) -> pd.DataFrame:
    df = df_fdp.copy()
    df["__API_KEY__"] = df["AIB_MARCA_Y_DESC_API"].astype(str).str.upper().str.strip()
    ex = expo.copy()
    ex["__API_KEY__"] = ex["Denominacion API"]

    df = df.merge(
        ex[["__API_KEY__","Denominacion API","Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]],
        how="left", on="__API_KEY__"
    ).drop(columns=["__API_KEY__"])

    df = df.rename(columns={
        "Denominacion API": "EXPOSICIÓN.Denominacion API",
        "Marca": "EXPOSICIÓN.Marca",
        "Tipo": "EXPOSICIÓN.Tipo",
        "Torque max": "EXPOSICIÓN.Torque max",
        "SENSIBILIDAD CR": "EXPOSICIÓN.SENSIBILIDAD CR",
        "SENSIBILIDAD EST": "EXPOSICIÓN.SENSIBILIDAD EST",
    })

    df["ID_tmp"] = (df["NOMBRE_CORTO"].fillna("").astype(str) + df["NIVEL_5"].fillna("").astype(str))
    df = df[(df["ID_tmp"]!="") & df["NOMBRE_CORTO"].notna()]
    df = df.drop_duplicates(subset=["ID_tmp"]).drop(columns=["ID_tmp"])

    for c in ["AIBRR_TORQUE_MAXIMO_REDUCTOR","AIBEB_TORQUE_MAXIMO_REDUCTOR"]:
        df[c] = df[c].fillna(0)

    df["NIVEL_2"] = df["NIVEL_2"].replace({"Tierra del Fuego": "Chubut"})
    return df

def attach_produccion(df: pd.DataFrame, prod: pd.DataFrame) -> pd.DataFrame:
    return df.merge(prod, how="left", on="NIVEL_4")

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["PROD_OIL_1"] = d["PROD_OIL_1"].replace({0: np.nan})
    d["Ratio_Produccion"] = d["PROD_OIL"] / d["PROD_OIL_1"]

    def seg_consecuencia(x):
        if pd.isna(x): return "0 - 0,0025"
        if x >= 0.0065: return "Mayor a 0,0065"
        if x >= 0.0025: return "0,0025 - 0,0065"
        return "0 - 0,0025"
    d["f_CONSECUENCIA"] = d["Ratio_Produccion"].apply(seg_consecuencia)
    d["CONSECUENCIA"] = d["f_CONSECUENCIA"]

    d["TORQUE_BAL[%]"] = d["AIBEB_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]
    d["TORQUE[%]"]     = d["AIBRR_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]

    d["f_CR"]  = np.select([d["TORQUE[%]"] > 1, d["TORQUE[%]"] >= 0.85], [10, 3.7], default=0)
    d["f_CR3"] = d["EXPOSICIÓN.SENSIBILIDAD CR"].map({"ALTA":3, "MEDIA":1.5}).fillna(1.0)
    d["CAJA REDUCTORA"]  = d["f_CR"] * d["f_CR3"]
    d["CAJA REDUCTORA_"] = np.where(d["TORQUE_BAL[%]"]>1, 1.5*d["CAJA REDUCTORA"], d["CAJA REDUCTORA"])

    d["f_estr"]  = np.select([d["ESTRUCTURA_PCT"] >= 0.95, d["ESTRUCTURA_PCT"] > 0.85], [20,5], default=1)
    d["f_estr3"] = d["EXPOSICIÓN.SENSIBILIDAD EST"].map({"ALTA":3,"MEDIA":1.5}).fillna(1.0)
    d["ESTRUCTURA"] = d["f_estr"] * d["f_estr3"]

    d["f_GPM"]  = np.select([d["AIB_GPM"] > 9, d["AIB_GPM"] > 7], [3, 1.5], default=1)
    d["f_GPM2"] = np.where(d["LLENAD_BOMBA_PCT"] < 0.75, d["f_GPM"]*2, d["f_GPM"])

    d["EXIGENCIA"]   = d["f_GPM2"] * d["ESTRUCTURA"] + d["CAJA REDUCTORA_"]
    d["F_EXIGENCIA"] = np.select([d["EXIGENCIA"]>10, d["EXIGENCIA"]>4], ["A","M"], default="B")

    def crit(row):
        rp = row["Ratio_Produccion"]; fe = row["F_EXIGENCIA"]
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "CRITICO"
        if rp >= 0.0025 and fe == "A": return "ALERTA"
        if rp <  0.0025 and fe == "A": return "ALERTA"
        if rp >= 0.0065 and fe == "M": return "ALERTA"
        if rp >= 0.0025 and fe == "M": return "ALERTA"
        if rp <  0.0025 and fe == "M": return "ALERTA"
        return "NORMAL"
    d["CRITICIDAD"] = d.apply(crit, axis=1).astype(str)
    d["ref_criticidad"] = d["CRITICIDAD"].map({"CRITICO":1, "ALERTA":2, "NORMAL":3}).astype("Int64")

    def color_detalle(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return ""
        if rp >= 0.0065 and fe == "A": return "RED"
        if rp >= 0.0025 and fe == "A": return "ORANGE"
        if rp <  0.0025 and fe == "A": return "ORANGE"
        if rp >= 0.0065 and fe == "M": return "YELLOW"
        if rp >= 0.0025 and fe == "M": return "YELLOW"
        if rp <  0.0025 and fe == "M": return "YELLOW"
        if rp >= 0.0065 and fe == "B": return "GREEN"
        if rp >= 0.0025 and fe == "B": return "GREEN"
        if rp <  0.0025 and fe == "B": return "GREEN"
        return ""
    d["Color Detalle CRITICIDAD"] = np.vectorize(color_detalle)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    d["criticidad color"] = d["CRITICIDAD"].map({"CRITICO":"Red", "ALERTA":"YELLOW", "NORMAL":"Green"}).fillna("White")
    d["criticidad texto"] = d["CRITICIDAD"].map({"CRITICO":"white", "ALERTA":"black", "NORMAL":"white"}).fillna("White")

    def detalle_crit(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "AA"
        if rp >= 0.0025 and fe == "A": return "AM"
        if rp <  0.0025 and fe == "A": return "AB"
        if rp >= 0.0065 and fe == "M": return "MA"
        if rp >= 0.0025 and fe == "M": return "MM"
        if rp <  0.0025 and fe == "M": return "MB"
        if rp >= 0.0065 and fe == "B": return "BA"
        if rp >= 0.0025 and fe == "B": return "BM"
        if rp <  0.0025 and fe == "B": return "BB"
        return "NORMAL"
    d["DETALLE CRITICIDAD"] = np.vectorize(detalle_crit)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    d["FECHA_HORA"] = pd.to_datetime(d["FECHA_HORA"], errors="coerce")
    max_por_pozo = d.groupby("NOMBRE_CORTO", dropna=False)["FECHA_HORA"].max().rename("FECHA_MAX").reset_index()
    d = d.merge(max_por_pozo, on="NOMBRE_CORTO", how="left")
    d["Días sin dina"] = (pd.Timestamp.today().normalize() - d["FECHA_MAX"].dt.normalize()).dt.days

    d["Correlativo Exigencia"] = d["F_EXIGENCIA"].map({"A":"10 +", "M":"04-10", "B":"0-04"}).fillna("")
    d["ID"] = d["f_CONSECUENCIA"].fillna("").astype(str) + d["Correlativo Exigencia"].fillna("").astype(str)

    d = d[(d["NOMBRE_CORTO"].notna()) & (d["NOMBRE_CORTO"].astype(str)!="")]

    return d

# ================== PIPELINE ==================
def main():
    print("Conectando a Oracle (cx_Oracle)…")
    with get_connection() as conn:
        print("Leyendo FDP_DINA…")
        fdp = load_fdp(conn)

        print("Leyendo Producción…")
        prod = load_produccion(conn)

        print("Leyendo ACCIONES…")
        acciones = load_acciones(conn)

    print("Leyendo Excel EXPOSICIÓN…")
    expo = load_exposicion(EXPO_PATH, EXPO_SHEET)

    print("Aplicando joins/limpieza (EXPOSICIÓN)…")
    fdp = join_expo(fdp, expo)

    print("Adjuntando Producción…")
    fdp = attach_produccion(fdp, prod)

    print("Calculando métricas / criticidad…")
    final = add_features(fdp)

    # -------- (1) Subset de columnas solicitadas --------
    columnas_matriz = [
        "NOMBRE_CORTO",
        "FECHA_HORA",
        "AIB_MARCA_Y_DESC_API",
        "PROD_OIL",
        "PROD_GAS",
        "PROD_WAT",
        "AIB_GPM",
        "MOTOR_DIAMETRO_POLEA",
        "EXPOSICIÓN.Denominacion API",
        "CRITICIDAD",
        "Días sin dina",
    ]
    faltantes = [c for c in columnas_matriz if c not in final.columns]
    if faltantes:
        raise KeyError(f"Faltan columnas en la matriz final: {faltantes}")
    matriz_subset = final[columnas_matriz].copy()

    # -------- (2) Exportar a Excel con 2 hojas --------
    out = Path("Matriz_AIB_export.xlsx")
    with pd.ExcelWriter(out, engine="openpyxl") as writer:
        matriz_subset.to_excel(writer, sheet_name="MATRIZ", index=False)
        acciones.to_excel(writer, sheet_name="ACCIONES", index=False)

    print(f"✅ Exportado {len(matriz_subset):,} filas (MATRIZ) + {len(acciones):,} filas (ACCIONES) → {out.absolute()}")

if __name__ == "__main__":
    main()


Conectando a Oracle (cx_Oracle)…
Leyendo FDP_DINA…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\2025247171.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)  # warning inofensivo


Leyendo Producción…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_41768\2025247171.py:105: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)  # warning inofensivo


Leyendo ACCIONES…
Leyendo Excel EXPOSICIÓN…
Hojas encontradas en 'C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx': ['Hoja5', 'EXPOSICIÓN', 'Hoja1', 'Hoja2', 'IDP', 'Hoja3', 'OBSERVACIONES']
Aplicando joins/limpieza (EXPOSICIÓN)…
Adjuntando Producción…
Calculando métricas / criticidad…
✅ Exportado 3,767 filas (MATRIZ) + 17,710 filas (ACCIONES) → C:\Users\ry16123\Matriz_AIB_export.xlsx


In [7]:
# -*- coding: utf-8 -*-
# Matriz de Criticidad de AIBs (fuera de Power BI) - cx_Oracle
# Hoja 1: columnas solicitadas de FDP_DINA transformada
# Hoja 2: consulta ACCIONES + columna ETIQUETA (clasificación por reglas + re-etiquetado)

from __future__ import annotations
from pathlib import Path
import re
import unicodedata
import numpy as np
import pandas as pd
import cx_Oracle

# ================== CREDENCIALES / RUTAS ==================
ORA_USER    = "RY16123"
ORA_PASS    = "Luciano284"
ORA_HOST    = "slplpgmoora03"
ORA_PORT    = 1527
ORA_SERVICE = "psfu"

EXPO_PATH   = r"C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx"
EXPO_SHEET  = None  # auto-detección de hoja

# ================== QUERIES ==================
SQL_FDP_DINA = """
SELECT DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_1,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_2,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4,
       DBU_FIC_ORG_ESTRUCTURAL.NIVEL_5,
       FIC_ULTIMO_DINAMOMETRO.FECHA_HORA,
       FIC_ULTIMO_DINAMOMETRO.REALIZADO_POR,
       FIC_ULTIMO_DINAMOMETRO.AIB_MARCA_Y_DESC_API,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBEB_TORQUE_MAXIMO_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.AIBRR_TORQUE_DISPONIBLE,
       FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100 AS LLENAD_BOMBA_PCT,
       FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100 AS ESTRUCTURA_PCT,
       VTOW_WELL_LAST_CONTROL_DET.PROD_OIL,
       VTOW_WELL_LAST_CONTROL_DET.PROD_GAS,
       VTOW_WELL_LAST_CONTROL_DET.PROD_WAT,
       FIC_ULTIMO_DINAMOMETRO.AIB_GPM,
       FIC_ULTIMO_DINAMOMETRO.AIB_DIAMETRO_POLEA_REDUCTOR,
       FIC_ULTIMO_DINAMOMETRO.MOTOR_DIAMETRO_POLEA
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
"""

SQL_PRODUCCION = """
SELECT UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4) AS NIVEL_4,
       SUM(VTOW_WELL_LAST_CONTROL_DET.PROD_OIL) AS PROD_OIL_1
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.VTOW_WELL_LAST_CONTROL_DET VTOW_WELL_LAST_CONTROL_DET,
     DISC_ADMINS.FIC_ULTIMO_DINAMOMETRO FIC_ULTIMO_DINAMOMETRO
WHERE ((VTOW_WELL_LAST_CONTROL_DET.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
       AND (FIC_ULTIMO_DINAMOMETRO.COMP_SK = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK))
  AND (DBU_FIC_ORG_ESTRUCTURAL.ESTADO = 'Produciendo')
  AND ((FIC_ULTIMO_DINAMOMETRO.AIBRE_SOLICITACION_DE_ESTRUCT/100) <> 0)
  AND ((FIC_ULTIMO_DINAMOMETRO.BBA_LLENADO_DE_BOMBA/100) <> 0)
  AND (DBU_FIC_ORG_ESTRUCTURAL.MET_PROD = 'Bombeo Mecánico')
GROUP BY UPPER(DBU_FIC_ORG_ESTRUCTURAL.NIVEL_4)
"""

SQL_ACCIONES = """
SELECT DISTINCT
  DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3,
  FIC_ACCIONES.ACTIVIDAD,
  FIC_ACCIONES.ESTADO,
  FIC_ACCIONES."ESTADO AUTORIZACIÓN",
  FIC_ACCIONES.FECHAACCION,
  FIC_ACCIONES."FECHA AUTORIZACION",
  FIC_ACCIONES.FECHAREALIZACION,
  FIC_ACCIONES.JUSTIFICACION,
  FIC_ACCIONES.OBJETIVO,
  FIC_ACCIONES.OBSERVACION,
  FIC_ACCIONES.ORIGEN,
  FIC_ACCIONES."INCREMENTO BRUTA",
  FIC_ACCIONES."INCREMENTO PETRÓLEO",
  FIC_ACCIONES.RECURSO,
  FIC_ACCIONES.SUBACTIVIDAD,
  FIC_ACCIONES."USUARIO AUTORIZANTE",
  FIC_ACCIONES."USUARIO CREADOR",
  DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_POZO,DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO,
  DBU_FIC_ORG_ESTRUCTURAL.NOMBRE_CORTO_POZO
FROM DISC_ADMINS.DBU_FIC_ORG_ESTRUCTURAL DBU_FIC_ORG_ESTRUCTURAL,
     DISC_ADMINS.FIC_ACCIONES FIC_ACCIONES
WHERE (FIC_ACCIONES.CLAVEPOZO = DBU_FIC_ORG_ESTRUCTURAL.COMP_SK)
  AND (DBU_FIC_ORG_ESTRUCTURAL.COMP_SK = FIC_ACCIONES.CLAVEPOZO)
  AND (FIC_ACCIONES.FECHAACCION >= TO_DATE('20230101000000','YYYYMMDDHH24MISS'))
  AND (DBU_FIC_ORG_ESTRUCTURAL.NIVEL_3 IN ('Las Heras CG - Canadon Escondida','Los Perales','El Guadal','Seco Leon - Pico Truncado'))
"""

# ================== CONEXIÓN ORACLE (cx_Oracle) ==================
def get_connection():
    dsn = cx_Oracle.makedsn(ORA_HOST, ORA_PORT, service_name=ORA_SERVICE)
    return cx_Oracle.connect(ORA_USER, ORA_PASS, dsn, encoding="UTF-8")

def read_sql_df(conn, sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, con=conn)

# ================== EXPO (auto-detección) ==================
def load_exposicion(path: str, sheet: str | None) -> pd.DataFrame:
    xls = pd.ExcelFile(path, engine="openpyxl")
    print(f"Hojas encontradas en '{path}': {xls.sheet_names}")

    if sheet and sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
    else:
        candidatos = ["EXPOSICION", "EXPOSICIÓN", "Exposicion", "Hoja1", "Sheet1"]
        df = None
        for s in candidatos:
            if s in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=s)
                break
        if df is None:
            for s in xls.sheet_names:
                tmp = pd.read_excel(xls, sheet_name=s)
                cols_norm = [str(c).strip().lower() for c in tmp.columns]
                if "denominacion api" in cols_norm:
                    df = tmp
                    print(f"Usando hoja detectada por columna: {s}")
                    break
        if df is None:
            raise ValueError(f"No pude encontrar la hoja de EXPOSICIÓN. Hojas: {xls.sheet_names}")

    if "Cantidad" in df.columns:
        df = df.drop(columns=["Cantidad"])

    # Normalizar columna clave
    col_api = None
    for c in df.columns:
        if str(c).strip().lower() == "denominacion api":
            col_api = c; break
    if col_api is None:
        raise ValueError("No encuentro la columna 'Denominacion API' en la hoja seleccionada.")

    df[col_api] = df[col_api].astype(str).str.upper().str.strip()
    df = df.rename(columns={col_api: "Denominacion API"})

    for c in ["Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]:
        if c not in df.columns:
            df[c] = np.nan
    return df

# ================== Utilidades NLP/Reglas ==================
_num = r"(\d+(?:[.,]\d+)?)"

MODEL_WORDS = (
    r"(lufkin|vulcan|siam|pump ?jack|wuel?fel|darco|maxii?|weatherford|"
    r"mel ?altium|altium|vc\d{3}|lm[-\s]?\d+|m\d{3}|c\d{3}|rm[-\s]?\d+)"
)

def _normalize(s):
    if s is None or (isinstance(s, float) and np.isnan(s)): return ""
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.lower()

def _to_float(x):
    try: return float(str(x).replace(",", "."))
    except: return None

def _any(txt, words):
    t = _normalize(txt)
    return any(w in t for w in words)

def _has_up(txt):   return _any(txt, ["increment", "aument", "subir", "sube", "suba", "+"])
def _has_down(txt): return _any(txt, ["bajar", "disminu", "reducir", "baja", "-"])

def _numbers_trend_up(txt):
    t = _normalize(txt)

    m = re.search(r"actual[^0-9]*"+_num+r".*final[^0-9]*"+_num, t)
    if m:
        a,f = _to_float(m.group(1)), _to_float(m.group(2))
        return None if a is None or f is None else f>a

    m = re.search(r"llevar[^0-9]*a[^0-9]*"+_num+r".*actual[^0-9]*"+_num, t)
    if m:
        target,actual = _to_float(m.group(1)), _to_float(m.group(2))
        return None if target is None or actual is None else target>actual

    m = re.search(r"sale[^0-9]*"+_num+r".*(entra|ingresa)[^0-9]*"+_num, t)
    if m:
        s,e = _to_float(m.group(2)), _to_float(m.group(3))
        return None if s is None or e is None else e>s

    m = re.search(r"\bs[^0-9:]*[: ]\s*"+_num+r".*\be[^0-9:]*[: ]\s*"+_num, t)
    if m:
        s,e = _to_float(m.group(1)), _to_float(m.group(2))
        return None if s is None or e is None else e>s

    m = re.search(_num+r"\s*(rpm|gpm|hz|spm)\D{0,15}(?:a|@|→|-?>)\D{0,15}"+_num, t)
    if m:
        a,f = _to_float(m.group(1)), _to_float(m.group(3))
        return None if a is None or f is None else f>a

    return None

def _polea_direction(txt):
    trend = _numbers_trend_up(txt)
    if trend is None: return None
    return "SUBE REGIMEN" if trend else "BAJA REGIMEN"

def classify_accion(objetivo, observacion) -> str:
    o   = _normalize(objetivo)
    c   = _normalize(observacion)
    txt = f"{o} {c}"

    # --- A) CAMBIO CARRERA (prioritario sobre contrapesar)
    if "carrera" in txt and re.search(r"\b(minim|maxim|intermed|bajar|subir|aumentar|disminuir|cambio de carrera|54|64|74|84|96|118|130|149|168|192)\b", txt):
        return "CAMBIO CARRERA"

    # --- B) CAMBIO AIB: verbos + AIB/modelo; “proveniente/locación/trasladar/tomar AIB”
    if re.search(r"\b(instalar|montar|desmontar|transportar|intercambio|enroque|cambio|tomar)\b.*\b(aib|"+MODEL_WORDS+r")\b", txt) \
       or re.search(r"\b(proveniente de|locaci[oó]n|trasladar|mover|llevar)\b.*\b(aib|"+MODEL_WORDS+r")\b", txt):
        return "CAMBIO AIB"

    # --- C) CONTRAPESAR “puro”
    if re.search(r"\b(mover|colocar|ubicar|retirar|balancear|balanceo|contrapesar)\b.*\b(contrapesos?|placas?)\b", txt):
        return "CONTRAPESAR"

    # --- D) SUBE/BAJA por evidencia numérica o de polea
    trend = _numbers_trend_up(txt)
    if trend is True:  base = "SUBE REGIMEN"
    elif trend is False: base = "BAJA REGIMEN"
    else: base = ""

    if base == "" and ("polea" in txt or "cambio de polea" in txt):
        polea = _polea_direction(txt)
        if polea: base = polea
        else:
            base = "SUBE REGIMEN" if ("optimizacion" in o or "optimización" in objetivo.lower()) else ("BAJA REGIMEN" if "operativa" in o else "SUBE REGIMEN")

    # verbos de subir/bajar (sin números claros)
    if base == "":
        if _has_up(txt)  and re.search(r"\b(regimen|gpm|rpm|hz|spm|mci|vsd|pid|pip)\b", txt):  base = "SUBE REGIMEN"
        if _has_down(txt) and re.search(r"\b(regimen|gpm|rpm|hz|spm|mci)\b", txt):             base = "BAJA REGIMEN"

    # --- E) Ajustes sin evidencia (llevar/estabilizar/ajustar/adecuar/setear X)
    if base == "" and re.search(r"\b(llevar|estabilizar|ajustar|adecuar|sete(ar|o))\b", txt) and re.search(r"\b(gpm|rpm|hz|spm|mci|vsd)\b", txt):
        base = "BAJA REGIMEN" if re.search(r"\b(golpe de fluido|gdf|gdf)\b", txt) else "SUBE REGIMEN"

    # --- F) ACONDICIONAR (superficie)
    if base == "":
        if re.search(r"\b(hot ?oil|ho\b|bache|batch|quimic|químic|dispersante|acido|ácido|solvente|freno|reparar freno|leuter|dispositivo|cubre ?polea|cubrepoleas?|rotador|medicion|medición|nivel|mf\b|nf\b|muestra|analisis|análisis|hermeticidad)\b", txt):
            base = "ACONDICIONAR EQUIP SUPERFICIE"

    # Guardrails
    if "optimizacion" in o or "optimización" in objetivo.lower():
        if base == "BAJA REGIMEN" or base == "": base = "SUBE REGIMEN"
    if "operativa" in o:
        strong_up = _has_up(txt) or trend is True or (_polea_direction(txt) == "SUBE REGIMEN")
        if base == "SUBE REGIMEN" and not strong_up:
            base = "BAJA REGIMEN"

    return base or "ACONDICIONAR EQUIP SUPERFICIE"

# ---- Re-etiquetado (capa post) ----
def _is_contrapesar(txt):
    return _any(txt, ["contrapes", "placa", "balancear", "balanceo", "manivela"])

def _is_carrera_action(txt):
    t = _normalize(txt)
    return ("carrera" in t) and _any(t, [
        "cambio", "cambiar", "llevar", "pasar", "disminuir", "aumentar",
        "minima", "mínima", "maxima", "máxima", "intermedia", "subir", "bajar"
    ])

def _is_surface_conditioning(txt):
    return _any(txt, [
        "hot oil"," ho","bache","batch","quimic","químic","dispersante",
        "acido","ácido","solvente","freno","leuter","dispositivo",
        "cubre","cubrepolea","cubre-polea","rotador","medicion","medición",
        "nivel"," mf"," nf","muestra","analisis","análisis","hermeticidad"
    ])

def _is_polea_change(txt):
    t = _normalize(txt)
    if "polea" not in t: return False
    return bool(re.search(r"(sale|entra|ingresa|cambio de polea|cambiar polea|s:|e:|montar polea)", t))

def _has_model_movement(txt):
    t = _normalize(txt)
    movement = re.search(r"\b(mover|trasladar|transportar|llevar)\b", t)
    model = re.search(r"\b(aib|"+MODEL_WORDS+r")\b", t)
    return bool(movement and model)

def reclassify_label(etiqueta_inicial: str, objetivo: str, observacion: str) -> str:
    """Aplica reglas de re-etiquetado sobre la etiqueta ya calculada (no cambia la lógica base)."""
    etq = (etiqueta_inicial or "").strip().upper()
    obj = _normalize(objetivo)
    obs = _normalize(observacion)
    txt = f"{obj} {obs}"

    is_opt = ("optimizacion" in obj or "optimización" in objetivo.lower())
    is_opr = ("operativa" in obj)

    # 1) Si es ACONDICIONAR EQUIP SUPERFICIE
    if etq == "ACONDICIONAR EQUIP SUPERFICIE":
        # 1.a) contrapesar siempre gana
        if _is_contrapesar(txt):
            return "CONTRAPESAR"
        # 1.b) cambio de polea: etiqueta por objetivo
        if _is_polea_change(txt):
            return "SUBE REGIMEN" if is_opt else "BAJA REGIMEN"
        # 1.c) movimiento + modelo → CAMBIO AIB
        if _has_model_movement(txt):
            return "CAMBIO AIB"
        # 1.d) GPM/RPM/Hz/SPM/MCI/VSD con verbos de ajuste (incluye setear)
        if _any(txt, ["gpm","rpm","hz","spm","mci","vsd"]) and _any(txt, ["llevar","estabilizar","dejar","ajustar","adecuar","setear","seteo"]):
            return "SUBE REGIMEN" if is_opt else "BAJA REGIMEN"
        return etq

    # 2) Si es BAJA REGIMEN
    if etq == "BAJA REGIMEN":
        if _is_contrapesar(txt):
            return "CONTRAPESAR"
        if _is_carrera_action(txt):
            return "CAMBIO CARRERA"
        if _is_surface_conditioning(txt) and not _is_polea_change(txt):
            return "ACONDICIONAR EQUIP SUPERFICIE"
        # cambio de polea domina: BAJA si Operativa, SUBE si Opti
        if _is_polea_change(txt):
            return "SUBE REGIMEN" if is_opt else "BAJA REGIMEN"
        return etq

    # 3) Si es CAMBIO AIB
    if etq == "CAMBIO AIB":
        # 3.a) “montar polea” (no AIB) y objetivo Optimización → SUBE
        if "montar polea" in txt and is_opt:
            return "SUBE REGIMEN"
        # 3.b) si NO hay verbos de AIB pero sí cambio de polea → por objetivo
        verbos_aib = re.search(r"\b(instalar|montar|desmontar|transportar|intercambio|enroque|tomar)\b", txt)
        if (not verbos_aib) and _is_polea_change(txt):
            dir_ = _polea_direction(txt)
            if dir_:
                return dir_
            return "SUBE REGIMEN" if is_opt else "BAJA REGIMEN"
        return etq

    # 4) Si es CAMBIO CARRERA
    if etq == "CAMBIO CARRERA":
        if re.search(r"\b(montar|instalar|transportar|trasladar|intercambio|proveniente|locaci[oó]n|enroque|tomar)\b", txt) and \
           _any(txt, ["aib","lufkin","vulcan","siam","pump jack","darco","weatherford","maxii","wuel","mel altium","altium"]):
            return "CAMBIO AIB"
        return etq

    # 5) Si es CONTRAPESAR
    if etq == "CONTRAPESAR":
        if _is_carrera_action(txt):
            return "CAMBIO CARRERA"
        return etq

    # 6) Si es SUBE REGIMEN
    if etq == "SUBE REGIMEN":
        if _is_carrera_action(txt):
            return "CAMBIO CARRERA"
        return etq

    return etq

# ================== Cargas base ==================
def load_fdp(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_FDP_DINA)
    df["NIVEL_4"] = df["NIVEL_4"].astype(str).str.upper()
    return df

def load_produccion(conn) -> pd.DataFrame:
    return read_sql_df(conn, SQL_PRODUCCION)

def load_acciones(conn) -> pd.DataFrame:
    df = read_sql_df(conn, SQL_ACCIONES)
    for c in ["FECHAACCION","FECHAREALIZACION","FECHA AUTORIZACION"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

# ================== Transformaciones ==================
def join_expo(df_fdp: pd.DataFrame, expo: pd.DataFrame) -> pd.DataFrame:
    df = df_fdp.copy()
    df["__API_KEY__"] = df["AIB_MARCA_Y_DESC_API"].astype(str).str.upper().str.strip()
    ex = expo.copy()
    ex["__API_KEY__"] = ex["Denominacion API"]

    df = df.merge(
        ex[["__API_KEY__","Denominacion API","Marca","Tipo","Torque max","SENSIBILIDAD CR","SENSIBILIDAD EST"]],
        how="left", on="__API_KEY__"
    ).drop(columns=["__API_KEY__"])

    df = df.rename(columns={
        "Denominacion API": "EXPOSICIÓN.Denominacion API",
        "Marca": "EXPOSICIÓN.Marca",
        "Tipo": "EXPOSICIÓN.Tipo",
        "Torque max": "EXPOSICIÓN.Torque max",
        "SENSIBILIDAD CR": "EXPOSICIÓN.SENSIBILIDAD CR",
        "SENSIBILIDAD EST": "EXPOSICIÓN.SENSIBILIDAD EST",
    })

    df["ID_tmp"] = (df["NOMBRE_CORTO"].fillna("").astype(str) + df["NIVEL_5"].fillna("").astype(str))
    df = df[(df["ID_tmp"]!="") & df["NOMBRE_CORTO"].notna()]
    df = df.drop_duplicates(subset=["ID_tmp"]).drop(columns=["ID_tmp"])

    for c in ["AIBRR_TORQUE_MAXIMO_REDUCTOR","AIBEB_TORQUE_MAXIMO_REDUCTOR"]:
        df[c] = df[c].fillna(0)

    df["NIVEL_2"] = df["NIVEL_2"].replace({"Tierra del Fuego": "Chubut"})
    return df

def attach_produccion(df: pd.DataFrame, prod: pd.DataFrame) -> pd.DataFrame:
    return df.merge(prod, how="left", on="NIVEL_4")

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["PROD_OIL_1"] = d["PROD_OIL_1"].replace({0: np.nan})
    d["Ratio_Produccion"] = d["PROD_OIL"] / d["PROD_OIL_1"]

    def seg_consecuencia(x):
        if pd.isna(x): return "0 - 0,0025"
        if x >= 0.0065: return "Mayor a 0,0065"
        if x >= 0.0025: return "0,0025 - 0,0065"
        return "0 - 0,0025"
    d["f_CONSECUENCIA"] = d["Ratio_Produccion"].apply(seg_consecuencia)
    d["CONSECUENCIA"] = d["f_CONSECUENCIA"]

    d["TORQUE_BAL[%]"] = d["AIBEB_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]
    d["TORQUE[%]"]     = d["AIBRR_TORQUE_MAXIMO_REDUCTOR"] / d["AIBRR_TORQUE_DISPONIBLE"]

    d["f_CR"]  = np.select([d["TORQUE[%]"] > 1, d["TORQUE[%]"] >= 0.85], [10, 3.7], default=0)
    d["f_CR3"] = d["EXPOSICIÓN.SENSIBILIDAD CR"].map({"ALTA":3, "MEDIA":1.5}).fillna(1.0)
    d["CAJA REDUCTORA"]  = d["f_CR"] * d["f_CR3"]
    d["CAJA REDUCTORA_"] = np.where(d["TORQUE_BAL[%]"]>1, 1.5*d["CAJA REDUCTORA"], d["CAJA REDUCTORA"])

    d["f_estr"]  = np.select([d["ESTRUCTURA_PCT"] >= 0.95, d["ESTRUCTURA_PCT"] > 0.85], [20,5], default=1)
    d["f_estr3"] = d["EXPOSICIÓN.SENSIBILIDAD EST"].map({"ALTA":3,"MEDIA":1.5}).fillna(1.0)
    d["ESTRUCTURA"] = d["f_estr"] * d["f_estr3"]

    d["f_GPM"]  = np.select([d["AIB_GPM"] > 9, d["AIB_GPM"] > 7], [3, 1.5], default=1)
    d["f_GPM2"] = np.where(d["LLENAD_BOMBA_PCT"] < 0.75, d["f_GPM"]*2, d["f_GPM"])

    d["EXIGENCIA"]   = d["f_GPM2"] * d["ESTRUCTURA"] + d["CAJA REDUCTORA_"]
    d["F_EXIGENCIA"] = np.select([d["EXIGENCIA"]>10, d["EXIGENCIA"]>4], ["A","M"], default="B")

    def crit(row):
        rp = row["Ratio_Produccion"]; fe = row["F_EXIGENCIA"]
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "CRITICO"
        if rp >= 0.0025 and fe == "A": return "ALERTA"
        if rp <  0.0025 and fe == "A": return "ALERTA"
        if rp >= 0.0065 and fe == "M": return "ALERTA"
        if rp >= 0.0025 and fe == "M": return "ALERTA"
        if rp <  0.0025 and fe == "M": return "ALERTA"
        return "NORMAL"
    d["CRITICIDAD"] = d.apply(crit, axis=1).astype(str)
    d["ref_criticidad"] = d["CRITICIDAD"].map({"CRITICO":1, "ALERTA":2, "NORMAL":3}).astype("Int64")

    def detalle_crit(rp, fe):
        if pd.isna(rp) or pd.isna(fe): return "NORMAL"
        if rp >= 0.0065 and fe == "A": return "AA"
        if rp >= 0.0025 and fe == "A": return "AM"
        if rp <  0.0025 and fe == "A": return "AB"
        if rp >= 0.0065 and fe == "M": return "MA"
        if rp >= 0.0025 and fe == "M": return "MM"
        if rp <  0.0025 and fe == "M": return "MB"
        if rp >= 0.0065 and fe == "B": return "BA"
        if rp >= 0.0025 and fe == "B": return "BM"
        if rp <  0.0025 and fe == "B": return "BB"
        return "NORMAL"
    d["DETALLE CRITICIDAD"] = np.vectorize(detalle_crit)(d["Ratio_Produccion"], d["F_EXIGENCIA"])

    d["FECHA_HORA"] = pd.to_datetime(d["FECHA_HORA"], errors="coerce")
    max_por_pozo = d.groupby("NOMBRE_CORTO", dropna=False)["FECHA_HORA"].max().rename("FECHA_MAX").reset_index()
    d = d.merge(max_por_pozo, on="NOMBRE_CORTO", how="left")
    d["Días sin dina"] = (pd.Timestamp.today().normalize() - d["FECHA_MAX"].dt.normalize()).dt.days

    d["Correlativo Exigencia"] = d["F_EXIGENCIA"].map({"A":"10 +", "M":"04-10", "B":"0-04"}).fillna("")
    d["ID"] = d["f_CONSECUENCIA"].fillna("").astype(str) + d["Correlativo Exigencia"].fillna("").astype(str)

    d = d[(d["NOMBRE_CORTO"].notna()) & (d["NOMBRE_CORTO"].astype(str)!="")]

    return d

# ================== PIPELINE ==================
def main():
    print("Conectando a Oracle (cx_Oracle)…")
    with get_connection() as conn:
        print("Leyendo FDP_DINA…")
        fdp = load_fdp(conn)

        print("Leyendo Producción…")
        prod = load_produccion(conn)

        print("Leyendo ACCIONES…")
        acciones = load_acciones(conn)

    print("Leyendo Excel EXPOSICIÓN…")
    expo = load_exposicion(EXPO_PATH, EXPO_SHEET)

    print("Aplicando joins/limpieza (EXPOSICIÓN)…")
    fdp = join_expo(fdp, expo)

    print("Adjuntando Producción…")
    fdp = attach_produccion(fdp, prod)

    print("Calculando métricas / criticidad…")
    final = add_features(fdp)

    # -------- (1) Subset de columnas solicitadas --------
    columnas_matriz = [
        "NOMBRE_CORTO",
        "FECHA_HORA",
        "AIB_MARCA_Y_DESC_API",
        "PROD_OIL",
        "PROD_GAS",
        "PROD_WAT",
        "AIB_GPM",
        "MOTOR_DIAMETRO_POLEA",
        "EXPOSICIÓN.Denominacion API",
        "CRITICIDAD",
        "Días sin dina",
    ]
    faltantes = [c for c in columnas_matriz if c not in final.columns]
    if faltantes:
        raise KeyError(f"Faltan columnas en la matriz final: {faltantes}")
    matriz_subset = final[columnas_matriz].copy()

    # -------- (2) Etiquetado de ACCIONES --------
    for col in ["OBJETIVO", "OBSERVACION"]:
        if col not in acciones.columns:
            acciones[col] = ""

    acciones["ETIQUETA_BASE"] = acciones.apply(
        lambda r: classify_accion(r.get("OBJETIVO", ""), r.get("OBSERVACION", "")),
        axis=1
    )
    acciones["ETIQUETA"] = acciones.apply(
        lambda r: reclassify_label(r.get("ETIQUETA_BASE", ""), r.get("OBJETIVO", ""), r.get("OBSERVACION", "")),
        axis=1
    )

    
        # ================== ACCIONES EN PROCESO / FINALIZADAS ==================
    # ================== ACCIONES EN PROCESO / FINALIZADAS ==================
    # Utilidades comunes
    def _plural_dia(n):
        try:
            n = int(n)
        except Exception:
            return f"{n} días"
        return f"{n} día" if n == 1 else f"{n} días"

    def _fila_texto(obs, etq, dias, msg="sin hacerse la acción"):
        obs = ("" if pd.isna(obs) else str(obs)).strip()
        etq = ("" if pd.isna(etq) else str(etq)).strip()
        dias_txt = _plural_dia(dias if dias is not pd.NA else 0)
        # sin punto final
        return f"{obs} -{etq} - Lleva {dias_txt} {msg}"

    def _acciones_texto_por_pozo(acciones_df: pd.DataFrame,
                                 matriz_df: pd.DataFrame,
                                 estados_validos: set,
                                 col_salida: str,
                                 msg_final: str = "sin hacerse la acción",
                                 add_finalizada: bool = False,
                                 date_tail: tuple[str, str, str] | None = None):
        """
        Devuelve DF [NOMBRE_CORTO, col_salida] enumerado por pozo.

        date_tail: (nombre_col_fecha, etiqueta_para_mostrar, formato_strftime)
                   p.ej. ("FECHAACCION", "FECHA ACCION", "%d/%m/%y")
                         ("FECHAREALIZACION", "FECHA REALIZACION", "%d/%m/%Y")
        """
        acc = acciones_df.copy()

        # Traer FECHA_HORA y CRITICIDAD desde MATRIZ
        mat_cols_needed = ["NOMBRE_CORTO", "FECHA_HORA", "CRITICIDAD"]
        mat_min = matriz_df[mat_cols_needed].drop_duplicates("NOMBRE_CORTO")
        acc = acc.merge(mat_min, on="NOMBRE_CORTO", how="left", suffixes=("", "_MATRIZ"))

        # Filtros base
        crit_mask  = acc["CRITICIDAD"].isin(["CRITICO", "ALERTA"])
        fecha_mask = (pd.to_datetime(acc["FECHA_HORA"], errors="coerce")
                      < pd.to_datetime(acc["FECHAACCION"], errors="coerce"))
        act_mask   = ~acc["ACTIVIDAD"].fillna("").str.contains("Intervención de Fondo", case=False, na=False)
        obj_mask   = (
            acc["OBJETIVO"].fillna("").str.contains("Operativa", case=False, na=False) |
            acc["OBJETIVO"].fillna("").str.contains("Optimización de producción", case=False, na=False)
        )

        acc_fil = acc[crit_mask & fecha_mask & act_mask & obj_mask].copy()

        # Estado
        acc_fil = acc_fil[acc_fil["ESTADO"].fillna("").str.upper().isin(estados_validos)].copy()
        if acc_fil.empty:
            return pd.DataFrame(columns=["NOMBRE_CORTO", col_salida])

        # Texto + días desde FECHAACCION
        today = pd.Timestamp.today().normalize()
        acc_fil["FECHAACCION"] = pd.to_datetime(acc_fil["FECHAACCION"], errors="coerce")
        acc_fil["dias_sin"] = (today - acc_fil["FECHAACCION"].dt.normalize()).dt.days.clip(lower=0).astype("Int64")

        # Para “y lleva XXXX días finalizada.” usamos FECHAREALIZACION
        if add_finalizada:
            acc_fil["FECHAREALIZACION"] = pd.to_datetime(acc_fil["FECHAREALIZACION"], errors="coerce")
            acc_fil["dias_fin"] = (today - acc_fil["FECHAREALIZACION"].dt.normalize()).dt.days.clip(lower=0).astype("Int64")
            acc_fil["__extra__"] = acc_fil["dias_fin"].apply(
                lambda d: f" y lleva {_plural_dia(0 if pd.isna(d) else d)} finalizada."
            )
        else:
            acc_fil["__extra__"] = ""

        # Sufijo de fecha (opcional) — p.ej. " FECHA ACCION: 13/08/25"
        if date_tail:
            col_fecha, etiqueta, fmt = date_tail
            col_fecha_norm = pd.to_datetime(acc_fil[col_fecha], errors="coerce")
            acc_fil["__fecha_tail__"] = col_fecha_norm.dt.strftime(fmt).fillna("")
            acc_fil["__fecha_tail__"] = acc_fil["__fecha_tail__"].apply(
                lambda s: f" {etiqueta}: {s}" if s else ""
            )
        else:
            acc_fil["__fecha_tail__"] = ""

        def _fila_texto(obs, etq, dias, extra="", fecha_tail=""):
            obs = ("" if pd.isna(obs) else str(obs)).strip()
            etq = ("" if pd.isna(etq) else str(etq)).strip()
            dias_txt = _plural_dia(dias if dias is not pd.NA else 0)
            return f"{obs} -{etq} - Lleva {dias_txt} {msg_final}.{extra}{fecha_tail}"

        acc_fil["__texto__"] = acc_fil.apply(
            lambda r: _fila_texto(r["OBSERVACION"], r["ETIQUETA"], r["dias_sin"],
                                  r["__extra__"], r["__fecha_tail__"]),
            axis=1
        )

        # Ordenar y enumerar
        acc_fil = acc_fil.sort_values(["NOMBRE_CORTO", "FECHAACCION"], ascending=[True, True])

        def _enumerar_series(s):
            lst = list(s)
            return "\n".join(f"{i}) {t}" for i, t in enumerate(lst, start=1))

        out = (
            acc_fil.groupby("NOMBRE_CORTO")["__texto__"]
            .apply(_enumerar_series)
            .reset_index(name=col_salida)
        )
        return out

    
    # ---- Construir y mergear columnas en MATRIZ ----
    
    # 1) ACCIONES EN PROCESO  -> agrega " FECHA ACCION: dd/mm/yy"
    estados_proceso = {"EN PROCESO", "BORRADOR", "NO INICIADA", "PROPUESTO"}
    agg_proceso = _acciones_texto_por_pozo(
        acciones, matriz_subset, estados_proceso, "ACCIONES EN PROCESO",
        msg_final="sin hacerse la acción",
        add_finalizada=False,
        date_tail=("FECHAACCION", "FECHA ACCION", "%d/%m/%y")
    )
    if agg_proceso.empty:
        agg_proceso = pd.DataFrame(columns=["NOMBRE_CORTO", "ACCIONES EN PROCESO"])
    matriz_subset = matriz_subset.merge(agg_proceso, on="NOMBRE_CORTO", how="left")
    matriz_subset["ACCIONES EN PROCESO"] = matriz_subset["ACCIONES EN PROCESO"].fillna("")

    # 2) ACCIONES FINALIZADAS -> agrega “y lleva XXXX días finalizada.” y “ FECHA REALIZACION dd/mm/YYYY”
    estados_finalizadas = {"FINALIZADA"}
    agg_final = _acciones_texto_por_pozo(
        acciones, matriz_subset, estados_finalizadas, "ACCIONES FINALIZADAS",
        msg_final="iniciada la acción",   # o “finalizada la acción” si preferís
        add_finalizada=True,
        date_tail=("FECHAREALIZACION", "FECHA REALIZACION", "%d/%m/%Y")
    )
    if agg_final.empty:
        agg_final = pd.DataFrame(columns=["NOMBRE_CORTO", "ACCIONES FINALIZADAS"])
    matriz_subset = matriz_subset.merge(agg_final, on="NOMBRE_CORTO", how="left")
    matriz_subset["ACCIONES FINALIZADAS"] = matriz_subset["ACCIONES FINALIZADAS"].fillna("")

    # ================== Columna VER ==================
    # Normalizamos textos (mayúsculas, NaN -> "")
    proc = matriz_subset["ACCIONES EN PROCESO"].fillna("").str.upper()

    # Soporta nombre singular o plural para finalizadas
    if "ACCIONES FINALIZADAS" in matriz_subset.columns:
        fin = matriz_subset["ACCIONES FINALIZADAS"].fillna("").str.upper()
    elif "ACCIONES FINALIZADA" in matriz_subset.columns:
        fin = matriz_subset["ACCIONES FINALIZADA"].fillna("").str.upper()
    else:
        fin = pd.Series("", index=matriz_subset.index)

    # 1) Proceso contiene: MATRIZ | CAMBIO AIB | CAMBIO CARRERA | CONTRAPESAR
    m1 = proc.str.contains(r"\bMATRIZ\b|CAMBIO AIB|CAMBIO CARRERA|CONTRAPESAR", regex=True)

    # 2) Proceso contiene: ACONDICIONAR EQUIP SUPERFICIE
    m2 = proc.str.contains("ACONDICIONAR EQUIP SUPERFICIE", regex=False)

    # 3) Finalizadas contiene: MATRIZ | CAMBIO AIB | CAMBIO CARRERA | CONTRAPESAR | ACONDICIONAR EQUIP SUPERFICIE
    m3 = fin.str.contains(r"\bMATRIZ\b|CAMBIO AIB|CAMBIO CARRERA|CONTRAPESAR|ACONDICIONAR EQUIP SUPERFICIE", regex=True)

    matriz_subset["VER"] = np.select(
        [m1, m2, m3],
        [
            "ACCION EN PROCESO PARA REGULARIZAR MATRIZ AIB",
            "ACONDICIONAR SUPERFICIE PARA TOMAR MEDICION",
            "ACTUALIZAR MEDICION DINAMOMETRICA",
        ],
        default=""
    )

 
    
    # -------- Exportar a Excel con 2 hojas --------
    out = Path("Matriz_AIB_export.xlsx")
    with pd.ExcelWriter(out, engine="openpyxl") as writer:
        matriz_subset.to_excel(writer, sheet_name="MATRIZ", index=False)
        acciones.to_excel(writer, sheet_name="ACCIONES", index=False)

    print(f"✅ Exportado {len(matriz_subset):,} filas (MATRIZ) + {len(acciones):,} filas (ACCIONES) → {out.absolute()}")


if __name__ == "__main__":
    main()


Conectando a Oracle (cx_Oracle)…
Leyendo FDP_DINA…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_33200\669960166.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo Producción…


C:\Users\ry16123\AppData\Local\Temp\ipykernel_33200\669960166.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, con=conn)


Leyendo ACCIONES…
Leyendo Excel EXPOSICIÓN…
Hojas encontradas en 'C:\Users\ry16123\Downloads\EXPOSICION-new 1.xlsx': ['Hoja5', 'EXPOSICIÓN', 'Hoja1', 'Hoja2', 'IDP', 'Hoja3', 'OBSERVACIONES']
Aplicando joins/limpieza (EXPOSICIÓN)…
Adjuntando Producción…
Calculando métricas / criticidad…
✅ Exportado 3,766 filas (MATRIZ) + 17,715 filas (ACCIONES) → C:\Users\ry16123\Matriz_AIB_export.xlsx
